In [19]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os

In [20]:
# Plot
plot_directory = 'result/pixel'

In [21]:
colors = [
	'#FF0000',
	'#00FFFF',
	'#0000FF',
	'#00008B',
	'#ADD8E6',
	'#800080',
	'#7FFFD4',
	'#008000',
	'#FF00FF',
	'#FFC0CB',
	'#C0C0C0',
	'#FFA500',
	'#000000',
	'#800000',
]

In [22]:
def load_json_file(file_path):
	try:
		with open(file_path, 'r') as file:
			data = json.load(file)
		return data
	
	except Exception as e:
		print(f"An error occurred while loading the JSON file: {e}")
		return None

In [23]:
def extract_fpss(metric_list):
	return list(metric_list[list(metric_list.keys())[0]][0]['metric'].keys())

In [24]:
def to_accuracy_vector(accuracy_result_seq, fpss, type='F1'):
	accuracy_vector = []
	for fps in fpss:
		accuracy_vector.append(accuracy_result_seq[fps][type])
	
	return accuracy_vector

In [25]:
def plot_scatter(xs, ys, x_label, y_label, title, fig_size=(8, 8)):
	plt.figure(figsize=fig_size)
	plt.scatter(xs, ys, c=colors[0], s=30, alpha=0.5)
	plt.title(title)
	plt.xlabel(x_label)
	plt.ylabel(y_label)
	plt.show()

In [26]:
def plot_scatter_label(xs, ys, labels, x_label, y_label, title, fig_size=(8, 8)):
	plt.figure(figsize=fig_size)
	for i in range(len(xs)):
		plt.scatter(xs[i], ys[i], c=colors[labels[i]], s=30, alpha=0.5)
	plt.title(title)
	plt.xlabel(x_label)
	plt.ylabel(y_label)
	plt.show()

In [27]:
def filter_list_index(input_list, indices_to_remove):
	return [item for i, item in enumerate(input_list) if i not in indices_to_remove]

In [28]:
def round_float_to_sigfigs(number, sigfigs):
	return round(number, sigfigs)

In [29]:
def round_floats_to_sigfigs(input_list, sigfigs):
	return [round(number, sigfigs) for number in input_list]

In [30]:
def normalize_list(input_list):
	# Check if the list is not empty
	if not input_list:
		return []

	min_val = min(input_list)
	max_val = max(input_list)

	# Handle the case where all elements in the list are the same
	if min_val == max_val:
		return [0.5] * len(input_list)

	return [(x - min_val) / (max_val - min_val) for x in input_list]


## Plot

In [31]:
FEATURE_TYPE = 'Pixel'

In [32]:
plot_filenames = sorted(os.listdir(plot_directory))
plot_video_names = sorted(list(set([f.split('_')[0] for f in plot_filenames])))

In [33]:
fpss = extract_fpss(load_json_file(os.path.join(plot_directory, plot_video_names[0] + f"_Accuracy_Result.json")))

In [34]:
accuracy_result = load_json_file(os.path.join(plot_directory, plot_video_names[0] + "_Accuracy_Result.json"))
# [clip_result['metric']['1']['F1'] for clip_result in accuracy_result['2']]

In [35]:
feature_result = load_json_file(os.path.join(plot_directory, plot_video_names[0] + f"_Feature_{FEATURE_TYPE}_Result.json"))
# [clip_result['feature']['1'] for clip_result in feature_result['2']]

In [36]:
for fps in fpss:
	print(f"FPS: {fps}")
	
	corr_list = []
	for v in plot_video_names:
		acc_list_video = []
		fea_list_video = []

		accuracy_result = load_json_file(os.path.join(plot_directory, v + "_Accuracy_Result.json"))
		feature_result = load_json_file(os.path.join(plot_directory, v + f"_Feature_{FEATURE_TYPE}_Result.json"))

		for class_idx in list(accuracy_result.keys()):
			acc_list_video_class = [clip_result['metric'][fps]['F1'] for clip_result in accuracy_result[class_idx]]
			fea_list_video_class = [clip_result['feature'][fps] for clip_result in feature_result[class_idx]]

			acc_list_video.extend(acc_list_video_class)
			fea_list_video.extend(fea_list_video_class)
		
		assert(len(acc_list_video) == len(fea_list_video))

		correlation_matrix = np.corrcoef(acc_list_video, normalize_list(fea_list_video))
		correlation_coefficient = correlation_matrix[0, 1]

		corr_list.append(correlation_coefficient)
	
	# print(f'Corrs Among All {len(plot_video_names)} Videos {corr_list}')
	print(f'Average Corr Among All {len(plot_video_names)} Videos {np.average(np.array(corr_list))}')


FPS: 1
Average Corr Among All 4 Videos 0.28935840388648115
FPS: 2
Average Corr Among All 4 Videos 0.3665193539379963
FPS: 3
Average Corr Among All 4 Videos 0.331027098490512
FPS: 5
Average Corr Among All 4 Videos 0.30820735215504563
FPS: 6
Average Corr Among All 4 Videos 0.29104631676969306
FPS: 10
Average Corr Among All 4 Videos 0.23177244949148762
FPS: 15
Average Corr Among All 4 Videos 0.1766602708485514
FPS: 30
Average Corr Among All 4 Videos 0.12398393265801204
